# Homework Module 01

 Rahma Hayuning Astuti

## Question 1

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there? 

Answer: 19

In [14]:
import pandas as pd

df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [15]:
print(len(df.columns))

19


*Answer*:19

## Question 2

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

answer: 42.59

In [16]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

print(df.duration.std())

42.59435124195458


# Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?


In [17]:
len(df)

3066766

In [ ]:
df_new = df[(df.duration >= 1) & (df.duration <= 60)]
len(df_new)

3009173

In [19]:
print(df['duration'].min(), df['duration'].max())


-29.2 10029.183333333332


In [20]:
fract = len(df_new) / len(df)
print(f"Fraction of records remaining: {fract:.4f}")


Fraction of records remaining: 0.9812


Answer: 98%

## Question 4

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
    Fit a dictionary vectorizer
    Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [21]:
df_new.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [24]:
cat = ['PULocationID', 'DOLocationID']
df_new[cat] = df_new[cat].astype(str)

from sklearn.feature_extraction import DictVectorizer
dicts = df_new[cat].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(dicts)
print(X_train.shape)

C:\Users\asus\AppData\Local\Temp/ipykernel_15168/3263877531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[cat] = df_new[cat].astype(str)


(3009173, 515)


answer: 515

## Question 5 

Now let's use the feature matrix from the previous step to train a model.

    Train a plain linear regression model with default parameters, where duration is the response variable
    Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [27]:
target = 'duration'
y_train = df_new[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
print(f'RMSE: {mean_squared_error(y_train, y_pred, squared=False)}')

RMSE: 7.649261929201487


answer: 7.64 

## Question 6

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [31]:
# df_train = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

len(df_val)

2913955

In [32]:
cat = ['PULocationID', 'DOLocationID']
df_new[cat] = df_new[cat].astype(str)

from sklearn.feature_extraction import DictVectorizer
dicts = df_new[cat].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(dicts)
print(X_train.shape)

C:\Users\asus\AppData\Local\Temp/ipykernel_15168/3263877531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[cat] = df_new[cat].astype(str)


(3009173, 515)


In [35]:
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [36]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
df_val[cat] = df_val[cat].astype(str)
dicts_val = df_val[cat].to_dict(orient='records')
X_val = dv.transform(dicts_val)


C:\Users\asus\AppData\Local\Temp/ipykernel_15168/3112560889.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val[cat] = df_val[cat].astype(str)


In [37]:
target = 'duration'
y_train = df_new[target].values
y_val = df_val[target].values

In [38]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
print(f'RMSE: {mean_squared_error(y_val, y_pred, squared=False)}')

RMSE: 7.811819793542861


Answer:7.81